In [2]:
# Importing libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models

In [3]:
# Defining function for data preprocessing
def preproc(norm,reshape):
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # Importing data set
    K = len(np.unique(y_train)) # Classes np.unique Find the unique elements of an array
    Ntr = x_train.shape[0] # Number of training data
    Nte = x_test.shape[0] # Number of testing data
    Din = x_train.shape[1]*x_train.shape[2]*x_train.shape[3] # CIFAR10
    if norm:
        # Normalize pixel values
        x_train, x_test = x_train / 255.0, x_test / 255.0
    
    # Centering pixel values
    mean_image = np.mean(x_train, axis=0)
    x_train = x_train - mean_image
    x_test = x_test - mean_image

    # One hot encoding the labels
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)

    # Flatterning the input images and changing the data type
    if reshape:
        x_train = np.reshape(x_train,(Ntr,Din))
        x_test = np.reshape(x_test,(Nte,Din))
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    print('x_train:', x_train.shape)
    print('x_test:', x_test.shape)
    print('y_train:', y_train.shape)
    print('y_test:', y_test.shape)
    return x_train,y_train,x_test,y_test

In [4]:
# Defining regularized loss function
def regloss(y_pred,y,w1,w2=0):
    batch_size=y_pred.shape[0] # Determining the number of input data
    loss=(1/(batch_size))*(np.square(y-y_pred)).sum()+reg*(np.sum(w1*w1)+np.sum(w2*w2))
    return loss

In [5]:
# Defining accuracy function
def accuracy(y_pred,y):
    batch_size=y_pred.shape[0] # Determining the number of input data
    K=y_pred.shape[1] # Determining number of classes
    acc=1-(1/(batch_size*K))*(np.abs(np.argmax(y,axis=1)-np.argmax(y_pred,axis=1))).sum()
    return acc

1.Linear Classifier

In [7]:
# Defining linear Classifier function
def linclas(x_train,y_train,x_test,y_test,lr,lr_decay,reg):
    Ntr = x_train.shape[0] # Number of training data
    Nte = x_test.shape[0] # Number of testing data
    Din = x_test.shape[1]
    K = y_test.shape[1]
    batch_size=x_train.shape[0]

    loss_history = []
    loss_history_test = []
    train_acc_history = []
    val_acc_history = []
    seed = 0
    rng = np.random.default_rng(seed=seed)

    # Initializing weight and bias arrays
    Din=x_train.shape[1]
    std=1e-5
    w1 = std*np.random.randn(Din, K)
    b1 = np.zeros(K)

    for t in range(iterations):
        # Shuffling the training data set to randomize the training process.To prevent overfitting
        indices = np.arange(Ntr)
        rng.shuffle(indices)
        x=x_train[indices]
        y=y_train[indices]

        # Forward pass
        y_pred=x.dot(w1)+b1
        y_pred_test=x_test.dot(w1)+b1

        # Calculating loss
        train_loss=regloss(y_pred,y,w1)
        test_loss=regloss(y_pred_test,y_test,w1)
        loss_history.append(train_loss)
        loss_history_test.append(test_loss)
        
        # Calculating accuracy
        train_acc=accuracy(y_pred,y)
        train_acc_history.append(train_acc)

        test_acc=accuracy(y_pred_test,y_test)
        val_acc_history.append(test_acc)

        if t%10 == 0:
            print('epoch %d/%d: loss= %f-- ,test loss= %f--,train accracy= %f--, test accracy= %f' % (t,iterations,train_loss,test_loss,train_acc,test_acc))

        # Backward pass
        dy_pred=(1./batch_size)*2.0*(y_pred-y)
        dw1=x.T.dot(dy_pred)+reg*w1
        db1=dy_pred.sum(axis=0)

        # Updating parameters
        w1-=lr*dw1
        b1-=lr*db1
        lr*=lr_decay
    return w1,b1,loss_history,loss_history_test,train_acc_history,val_acc_history

# Defining parameters and running the linear classifier
iterations = 300
lr = 1.4e-2
lr_decay= 0.999
reg = 5e-6 # Lamda(regularization constant for the loss function)
x_train,y_train,x_test,y_test=preproc(norm=True,reshape=True)
w1,b1,loss_history,loss_history_test,train_acc_history,val_acc_history=linclas(x_train,y_train,x_test,y_test,lr,lr_decay,reg)

x_train: (50000, 3072)
x_test: (10000, 3072)
y_train: (50000, 10)
y_test: (10000, 10)
epoch 0/300: loss= 1.000004-- ,test loss= 1.000000--,train accracy= 0.663628--, test accracy= 0.662950
epoch 10/300: loss= 0.876638-- ,test loss= 0.876234--,train accracy= 0.756446--, test accracy= 0.757950
epoch 20/300: loss= 0.842541-- ,test loss= 0.842415--,train accracy= 0.762634--, test accracy= 0.767740
epoch 30/300: loss= 0.823617-- ,test loss= 0.823713--,train accracy= 0.766444--, test accracy= 0.770400
epoch 40/300: loss= 0.812389-- ,test loss= 0.812705--,train accracy= 0.768580--, test accracy= 0.771520
epoch 50/300: loss= 0.805426-- ,test loss= 0.805967--,train accracy= 0.770354--, test accracy= 0.772940
epoch 60/300: loss= 0.800906-- ,test loss= 0.801676--,train accracy= 0.771518--, test accracy= 0.773440
epoch 70/300: loss= 0.797822-- ,test loss= 0.798822--,train accracy= 0.772712--, test accracy= 0.773610
epoch 80/300: loss= 0.795605-- ,test loss= 0.796832--,train accracy= 0.773842--, te